In [4]:
import numpy as np

# Eliminación Gaussiana

En todas las implementaciones, observar el argumento por default en el llamado a la función

Implementación usando vectorización de Numpy

In [5]:
def eliminacion_gaussiana(A,b,resultados_parciales=False):
    '''
    Esta función calcula la solución de un sistema de ecuaciones Ax=b usando Eliminación Gaussiana para llevar la 
    matriz A a su forma triangular superior y después realiza la sustitución hacia atrás.
    '''
    n = A.shape[0]
    assert b.shape[0] == n  # Aquí nos aseguramos que la forma de b sea compatible con la forma de A
    A = np.hstack((A,b.reshape(-1,1))) # Formamos la matriz aumentada del sistema
    if resultados_parciales: print("Inicial:\n", A) # Podemos escribir el if en una sola línea
    for k in range(n-1): # Recorremos cada columna
        for i in range(k+1,n): # Recorremos de la diagonal hacia adelante
            ratio = A[i,k]/A[k,k]
            A[i] = A[i]-ratio*A[k] # En este punto usamos la vectorización
            if resultados_parciales: print(A)
    if resultados_parciales: print("Forward elimination:\n",A)
    variables = A[:,n]
    for k in reversed(range(n)): # Realizamos la sustitución hacia atras
        for j in range(k+1,n):
            variables[k] = variables[k] - A[k,j]*variables[j] # Otra vez, usamos vectorización
        variables[k] = variables[k]/A[k,k]
    return variables

Ahora, incluyamos el pivoteo parcial. Esto lo hacemos manipulando los índices de los arreglos de numpy.

In [6]:
def eliminacion_gaussiana_pp(A,b,resultados_parciales=False):
    '''
    Esta función calcula la solución de un sistema de ecuaciones Ax=b usando Eliminación Gaussiana con
    pivoteo parcial para llevar la matriz A a su forma triangular superior y después realiza la 
    sustitución hacia atrás.
    '''
    n = A.shape[0]
    assert b.shape[0] == n
    A = np.hstack((A,b.reshape(-1,1))) # Formamos la matriz aumentada del sistema
    if resultados_parciales: print("Inicial:\n", A)
    for k in range(n-1):
        idx_max = np.argmax(np.abs(A[k:,k])) + k
        new_idxs = list(range(A.shape[0]))
        new_idxs[k] = idx_max
        new_idxs[idx_max] = k
        A = A[new_idxs]
        if resultados_parciales: print("cambio de renglones:\n",A)
        for i in range(k+1,n):
            ratio = A[i,k]/A[k,k]
            A[i] = A[i]-ratio*A[k] # En este punto usamos la vectorización
            if resultados_parciales: print(A)
    if resultados_parciales: print("Forward elimination:\n",A)
    variables = A[:,n]
    for k in reversed(range(n)):
        for j in range(k+1,n):
            variables[k] = variables[k] - A[k,j]*variables[j] # Otra vez, usamos vectorización
        variables[k] = variables[k]/A[k,k]
    return variables

## Aspectos importantes

¿Cómo juntamos arreglos en uno solo?

In [6]:
A = np.array([[3,2,1],[5,3,4],[1,1,-1]],dtype=float)
b = np.array([1,2,1],dtype=float)

print(A)
print(b.reshape(-1,1))
np.hstack((A,b.reshape(-1,1)))

[[ 3.  2.  1.]
 [ 5.  3.  4.]
 [ 1.  1. -1.]]
[[1.]
 [2.]
 [1.]]


array([[ 3.,  2.,  1.,  1.],
       [ 5.,  3.,  4.,  2.],
       [ 1.,  1., -1.,  1.]])

¿Cómo intercambiamos dos líneas de un arreglo?

In [10]:
A = np.array([[3,2,1],[5,3,4],[1,1,-1]],dtype=float)
print(A)
new_idxs = [2,1,0]
A = A[new_idxs]  # Aquí hacemos la permutación
print(A)

[[ 3.  2.  1.]
 [ 5.  3.  4.]
 [ 1.  1. -1.]]
[[ 1.  1. -1.]
 [ 5.  3.  4.]
 [ 3.  2.  1.]]


## Ejemplo 1

### Usando nuestra implementación

In [32]:
A = np.array([[3,2,1],[5,3,4],[1,1,-1]],dtype=float)
b = np.array([1,2,1],dtype=float)
print(A,"\n\n",b)

[[ 3.  2.  1.]
 [ 5.  3.  4.]
 [ 1.  1. -1.]] 

 [1. 2. 1.]


In [33]:
eliminacion_gaussiana(A,b)

array([-4.,  6.,  1.])

In [38]:
eliminacion_gaussiana_pp(A,b,resultados_parciales=True)

Inicial:
 [[ 3.  2.  1.  1.]
 [ 5.  3.  4.  2.]
 [ 1.  1. -1.  1.]]
cambio de renglones:
 [[ 5.  3.  4.  2.]
 [ 3.  2.  1.  1.]
 [ 1.  1. -1.  1.]]
[[ 5.   3.   4.   2. ]
 [ 0.   0.2 -1.4 -0.2]
 [ 1.   1.  -1.   1. ]]
[[ 5.   3.   4.   2. ]
 [ 0.   0.2 -1.4 -0.2]
 [ 0.   0.4 -1.8  0.6]]
cambio de renglones:
 [[ 5.   3.   4.   2. ]
 [ 0.   0.4 -1.8  0.6]
 [ 0.   0.2 -1.4 -0.2]]
[[ 5.   3.   4.   2. ]
 [ 0.   0.4 -1.8  0.6]
 [ 0.   0.  -0.5 -0.5]]
Forward elimination:
 [[ 5.   3.   4.   2. ]
 [ 0.   0.4 -1.8  0.6]
 [ 0.   0.  -0.5 -0.5]]


array([-4.,  6.,  1.])

### Usando implementaciones de Python

Usando numpy.

In [15]:
import numpy as np

np.linalg.solve(A,b)

array([-4.,  6.,  1.])

In [16]:
np.linalg.inv(A)@b

array([-4.,  6.,  1.])

Usando [`scipy`](https://docs.scipy.org/doc/scipy/tutorial/index.html#user-guide)

In [24]:
import scipy

scipy.linalg.solve(A,b)

array([-4.,  6.,  1.])

En el siguiente ejemplo vemos que, en general, necesitamos el método con pivoteo parcial

In [ ]:
A = np.array([[0,2,1],[5,0,4],[1,1,-1]],dtype=float)
b = np.array([1,2,1],dtype=float)
print(A,"\n\n",b,"\n")

print(f"Eliminación Gaussiana sin pivoteo parcial\n{eliminacion_gaussiana(A,b)}\n")
print(f"Eliminación Gaussiana con pivoteo parcial\n{eliminacion_gaussiana_pp(A,b)}\n")
print(f"Usando el solver de numpy\n{np.linalg.solve(A,b)}")

Comparemos el tiempo de ejecución de ambas opciones (la propia y la de Python):

In [42]:
import time
import scipy

# --- opción 1 ---
inicio = time.time()
sol = eliminacion_gaussiana(A,b)
final = time.time()
print(f"Duración (EG): {final-inicio} segundos")

# --- opción 2 ---
inicio = time.time()
sol = eliminacion_gaussiana_pp(A,b)
final = time.time()
print(f"Duración (EGpp): {final-inicio} segundos")

# --- opción 3 ---
inicio = time.time()
sol = np.linalg.solve(A,b)
final = time.time()
print(f"Duración (numpy): {final-inicio} segundos")

# --- opción 4 ---
inicio = time.time()
sol = scipy.linalg.solve(A,b)
final = time.time()
print(f"Duración (scipy): {final-inicio} segundos")

Duración: 0.0003113746643066406 segundos
Duración: 0.00017833709716796875 segundos
Duración: 8.392333984375e-05 segundos
Duración: 0.0008668899536132812 segundos


Ahora, veamos con un ejemplo grande

In [20]:
import numpy as np

np.random.seed(22)
Ab = np.random.normal(size=(200,201))

A = Ab[:,:-1].copy()
b = Ab[:,-1].copy()
print(f"Determinante de A: {np.linalg.det(A)}")

Determinante de A: 1.7298866403633389e+187


In [21]:
import time
import scipy

# --- opción 2 ---
inicio = time.time()
sol = eliminacion_gaussiana_pp(A,b)
final = time.time()
print(f"Duración (EGpp): {final-inicio} segundos")

# --- opción 3 ---
inicio = time.time()
sol = np.linalg.solve(A,b)
final = time.time()
print(f"Duración (numpy): {final-inicio} segundos")

# --- opción 4 ---
inicio = time.time()
sol = scipy.linalg.solve(A,b)
final = time.time()
print(f"Duración (scipy): {final-inicio} segundos")

Duración (EGpp): 0.1434800624847412 segundos
Duración (numpy): 0.000736236572265625 segundos
Duración (scipy): 0.0961148738861084 segundos


# Método de Simpson para Integración